In [6]:
pip install pygam

  Using cached pygam-0.9.0-py3-none-any.whl (522 kB)
  Using cached progressbar2-4.3.2-py3-none-any.whl (50 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pygam import LinearGAM
from scipy.optimize import minimize

%matplotlib inline

In [22]:
nb_actifs = 20
nb_dates = 256 * 5

idx = pd.Index(np.arange(nb_actifs))

mu = pd.Series(0.05,index=idx)
vols = pd.Series(0.2,index=idx)
correl = 0.7

covar = np.diag(vols) @ pd.DataFrame(correl * np.ones((nb_actifs,nb_actifs)) + (1-correl) * np.eye(nb_actifs),index=idx,columns=idx) @ np.diag(vols)
A = np.linalg.cholesky(covar)

rdts_observes = mu/256 + pd.DataFrame(np.random.randn(nb_dates,nb_actifs)) @ A.T / 16


In [23]:
mu_estime = rdts_observes.mean() * 256
covar_estimee = rdts_observes.cov() * 256


In [26]:
## cette fonction objectif permet de calculer 1) le rdt prédit à partir de 

def objective(weights, returns=rdts_observes):
    # Appliquer les régressions splines adaptatives pour modéliser la relation entre les rendements et les poids
    splines_model = LinearGAM().fit(returns,weights)
    predicted_returns = splines_model.predict(returns)

    return predicted_returns

# Contrainte : la somme des poids doit être égale à 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Contrainte : les poids doivent être dans la plage [0, 1]
bounds = tuple((0, 1) for asset in range(len(rdts_observes.columns)))

# Supposons que nous avons 20 actifs dans le portefeuille
initial_weights = np.ones(20) / 20  # Répartition égale au départ

# Utiliser la fonction minimize pour trouver les poids optimaux
result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Les poids optimaux se trouvent dans result.x
optimal_weights = result.x

print("Optimal Weights:", optimal_weights)


ValueError: Inconsistent input and output data shapes. found X: (1280, 20) and y: (20,)

In [ ]:
alloc_optimale_in_sample = pd.DataFrame(np.linalg.inv(covar_estimee),index=idx,columns=idx) @ mu_estime

In [ ]:
pnl_in_sample = rdts_observes @ alloc_optimale_in_sample
print('Sharpe = ' + str(pnl_in_sample.mean()/pnl_in_sample.std()*16))
pnl_in_sample.cumsum().plot()